In [41]:
import pandas as pd
import numpy as np
import math

In [42]:
bridges = pd.read_excel('../data/BMMS_overview.xlsx')

In [43]:
bridges = bridges[(bridges['road'].str.contains(pat = ('N1'), regex = False)) | (bridges['road'].str.contains(pat = ('N2'), regex = False))]

In [44]:
# Define bridge length bins and labels
length_bins = [0, 10, 50, 200, math.inf]
length_labels = ['S', 'M', 'L', 'XL']

# Categorize bridges per length class
bridges["length_class"] = pd.cut(bridges["length"], bins=length_bins, include_lowest=False, right=False, labels=length_labels)

In [45]:
# Keep only useful columns
keep_columns = ['road', 'lat', 'lon', 'length', 'condition', 'name', 'LRPName', 'chainage', 'km']
bridges = bridges[keep_columns]

In [46]:
# Filters out all variations of (R), (Right), (  R  ), etc.
patternDel = "\(\s*[a-zA-Z]*(R|r)[a-zA-Z]*\s*\)"
filter = bridges['name'].str.contains(patternDel)
bridges = bridges[~filter]

/Users/timofrazer/opt/anaconda3/lib/python3.8/site-packages/pandas/core/strings/accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [47]:
# Calculate distance between bridges
bridges1 = bridges.copy()
bridges1["dist"] = bridges1["km"].diff()

# Drop all bridges that are closer to the previous bridge then their own length
bridges = bridges.loc[bridges1["dist"]*1000 >= bridges["length"]]

## Convert to example data format

In [49]:
bridges['model_type'] = 'bridge'

In [51]:
roads = pd.read_csv('../data/_roads3.csv')
roads = roads[["road", "chainage", "lrp", "lat", "lon", "name"]]
length = (roads['chainage'] - roads['chainage'].shift(-1)).abs()
roads['length'] = (length * 1000)
roads.drop(roads.tail(1).index, inplace=True)
roads.reset_index()
roads['model_type'] = 'link'

FileNotFoundError: [Errno 2] No such file or directory: '../data/_roads3.csv'

In [50]:
bridges.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1763 entries, 1 to 19507
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   road        1763 non-null   object 
 1   lat         1763 non-null   float64
 2   lon         1763 non-null   float64
 3   length      1763 non-null   float64
 4   condition   1763 non-null   object 
 5   name        1763 non-null   object 
 6   LRPName     1763 non-null   object 
 7   chainage    1763 non-null   float64
 8   km          1763 non-null   float64
 9   model_type  1763 non-null   object 
dtypes: float64(5), object(5)
memory usage: 151.5+ KB
